In [10]:
# Auto reload modules
%load_ext autoreload
%autoreload 2

# Set CUDA_VISIBLE_DEVICES

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# matplotlib setting
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams["font.size"] = 14
%matplotlib inline


import numpy as np
from functools import partial

import torch
from torch import nn as nn

from itertools import repeat
import collections.abc

import torchvision.models as models
import copy
 
# from utils.model_trainer_generate import *
from utils.aggregate_block.model_trainer_generate import generate_cls_model

from collapsible_MLP import *
from MG_utils import get_collapsible_model, get_model_collapsible_slopes, collapse_model, get_model_size, get_num_parameters

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## initialize the regular model

In [2]:
regular_model = generate_cls_model("vit_b_16", )

## Make collapsable model with the appropriate fraction value

In [3]:
collapsible_model = get_collapsible_model(regular_model, fraction=0.3)

Collapsing layer 1.encoder.layers.encoder_layer_11.mlp
Collapsing layer 1.encoder.layers.encoder_layer_10.mlp
Collapsing layer 1.encoder.layers.encoder_layer_9.mlp
Collapsing layer 1.encoder.layers.encoder_layer_8.mlp


## load file

In [4]:
collapsible_model.load_state_dict(torch.load("/home/soheil/trojan/BackdoorBench-LayerCollapse/record/trojannn_0_1_ViT/defense/ft/defense_result.pt")['model'])

<All keys matched successfully>

## get the slopes and continue with your experiments

In [5]:
get_model_collapsible_slopes(collapsible_model, fraction=0.3)

1.encoder.layers.encoder_layer_11.mlp 0.9999831914901733
1.encoder.layers.encoder_layer_10.mlp 1.0000500679016113
1.encoder.layers.encoder_layer_9.mlp 1.0001270771026611


## To get model size:

In [6]:
print(f"model size before collapsing (GB): {get_model_size(regular_model)/1e9}")

model size before collapsing (GB): 2.745803072


## Collapse model

In [7]:
collapse_model(collapsible_model, fraction=0.3)

Collapsing layer 1.encoder.layers.encoder_layer_11.mlp
Collapsing layer 1.encoder.layers.encoder_layer_10.mlp
Collapsing layer 1.encoder.layers.encoder_layer_9.mlp


In [8]:
print(f"model size after collapsing (GB): {get_model_size(collapsible_model)/1e9}")

model size after collapsing (GB): 2.349146464


In [13]:
print(f"num parameters before collapsing: {get_num_parameters(regular_model)/1e6}M")
print(f"num parameters after collapsing: {get_num_parameters(collapsible_model)/1e6}M")

num parameters before collapsing: 85.806346M
num parameters after collapsing: 73.410827M


In [14]:
1 - get_num_parameters(collapsible_model)/get_num_parameters(regular_model)

0.14445923381937276